# Chicago Data Analysis with SQL

## 📌 Project Overview
In this project, I applied SQL queries to analyze real-world datasets from the **City of Chicago Data Portal**.  
The datasets cover socioeconomic indicators, public schools, and crime data.  
The project demonstrates SQL skills such as filtering, aggregation, joins, and analytical querying.  

---

## 📊 Datasets Used

1. **Socioeconomic Indicators in Chicago**  
   - Contains socioeconomic data by community area.  
   - Features include: hardship index, per capita income, percent unemployment, education levels, etc.  

2. **Chicago Public Schools**  
   - Information about school locations, performance ratings, and other key metrics.  
   - Useful for analyzing education quality and its relation to socioeconomic indicators.  

3. **Chicago Crime Data**  
   - Reported incidents of crime in the city of Chicago.  
   - Includes type of crime, location, and date.  


#### **Key skills learned**: SQL querying, data cleaning and preprocessing, pandas DataFrame manipulation, database integration with SQLite, data visualization, working with multiple datasets in Python.**bold text**

In [1]:
!pip install pandas

##### import sqlite3 as needed to run sql commands as well as prettytable

In [39]:
import csv, sqlite3
import prettytable

prettytable.DEFAULT = 'DEFAULT'

#creating the connection and making cursor
con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

##### Load the SQL magic module allows us to use sqlite3

In [40]:
!pip install ipython-sql
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


##### as our files are locally downloaded we will import them using import library

In [16]:
from google.colab import files
uploaded = files.upload()


Saving ChicagoCrimeData.csv to ChicagoCrimeData.csv
Saving ChicagoPublicSchools.csv to ChicagoPublicSchools.csv
Saving ChicagoCensusData.csv to ChicagoCensusData (1).csv


##### now load the files using read_csv and write them as sql using to_sql

In [41]:
import pandas as pd
CENSUS_DATA = "ChicagoCensusData.csv"
CHICAGO_PUBLIC_SCHOOLS= "ChicagoPublicSchools.csv"
CHICAGO_CRIME_DATA = "ChicagoCrimeData.csv"

df1 = pd.read_csv(CENSUS_DATA)
df2 = pd.read_csv(CHICAGO_PUBLIC_SCHOOLS)
df3 = pd.read_csv(CHICAGO_CRIME_DATA)

df1.to_sql("CENSUS_DATA", con, if_exists="replace", index=False)
df2.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists="replace", index=False)
df3.to_sql("CHICAGO_CRIME_DATA", con, if_exists="replace", index=False)

533

In [42]:
%sql sqlite:///FinalDB.db #This is the connection string for SQLite.

----------------------

### Viweing Columns

##### we will view thw columns of each tables for better analysing

In [52]:
df = %sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CENSUS_DATA');
df=pd.DataFrame(df)
df

 * sqlite:///FinalDB.db
Done.


,name,type,length(type)
0,COMMUNITY_AREA_NUMBER,REAL,4
1,COMMUNITY_AREA_NAME,TEXT,4
2,PERCENT_OF_HOUSING_CROWDED,REAL,4
3,PERCENT_HOUSEHOLDS_BELOW_POVERTY,REAL,4
4,PERCENT_AGED_16__UNEMPLOYED,REAL,4
5,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,REAL,4
6,PERCENT_AGED_UNDER_18_OR_OVER_64,REAL,4
7,PER_CAPITA_INCOME,INTEGER,7
8,HARDSHIP_INDEX,REAL,4


In [53]:
df=%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS');
df=pd.DataFrame(df)
df

 * sqlite:///FinalDB.db
Done.


,name,type,length(type)
0,School_ID,INTEGER,7
1,NAME_OF_SCHOOL,TEXT,4
2,"Elementary, Middle, or High School",TEXT,4
3,Street_Address,TEXT,4
4,City,TEXT,4
...,...,...,...
73,COMMUNITY_AREA_NUMBER,INTEGER,7
74,COMMUNITY_AREA_NAME,TEXT,4
75,Ward,INTEGER,7
76,Police_District,INTEGER,7


In [54]:
df = %sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_CRIME_DATA');
df=pd.DataFrame(df)
df

 * sqlite:///FinalDB.db
Done.


,name,type,length(type)
0,ID,INTEGER,7
1,CASE_NUMBER,TEXT,4
2,DATE,TEXT,4
3,BLOCK,TEXT,4
4,IUCR,TEXT,4
5,PRIMARY_TYPE,TEXT,4
6,DESCRIPTION,TEXT,4
7,LOCATION_DESCRIPTION,TEXT,4
8,ARREST,INTEGER,7
9,DOMESTIC,INTEGER,7


-------------------------------------

## Problems

Now as we have imported the data we will run some queries to extract the data

### 1. Census Data
#####  Community area names and numbers with per capita income less than 11000.

In [55]:
df= %sql select COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER from CENSUS_DATA where PER_CAPITA_INCOME<11000 ;
df=pd.DataFrame(df)
df

 * sqlite:///FinalDB.db
Done.


,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
0,West Garfield Park,26.0
1,South Lawndale,30.0
2,Fuller Park,37.0
3,Riverdale,54.0


##### 5 community areas with highest % of households below poverty line

In [58]:
df=%sql select COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA order by PERCENT_HOUSEHOLDS_BELOW_POVERTY desc limit 5;
df=pd.DataFrame(df)
df

 * sqlite:///FinalDB.db
Done.


,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,Riverdale,56.5
1,Fuller Park,51.2
2,Englewood,46.6
3,North Lawndale,43.1
4,East Garfield Park,42.4


###### Using a sub-query to find the name of the community area with highest hardship index

In [59]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX = ( SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


##### Using a sub-query to determine the Community Area Name with most number of crimes?

In [60]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = ( SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1);


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin


### 2. CHICAGO PUBLIC SCHOOLS Data

##### Listing the type of schools along with the average safety score for each type.

In [63]:
df = %sql select "Elementary, Middle, or High School" as type ,avg(SAFETY_SCORE) as averase_score from CHICAGO_PUBLIC_SCHOOLS group by "Elementary, Middle, or High School";
df=pd.DataFrame(df)
df

 * sqlite:///FinalDB.db
Done.


,type,averase_score
0,ES,49.520384
1,HS,49.623529
2,MS,48.000000


##### select and present the name of top 5 schools which have the highest graduation rate.

In [100]:
#as our data contains some object type values(NDA) so first we have to replace them
df = %sql SELECT NAME_OF_SCHOOL	, Graduation_Rate__ FROM CHICAGO_PUBLIC_SCHOOLS;
df = pd.DataFrame(df)
df['Graduation_Rate__'] = df['Graduation_Rate__'].replace("NDA", "0")
df = df.sort_values(by='Graduation_Rate__', ascending=False)
df.head()


 * sqlite:///FinalDB.db
Done.


,NAME_OF_SCHOOL,Graduation_Rate__
408,Northside College Preparatory High School,97.6
518,Walter Payton College Preparatory High School,96.9
531,Whitney M Young Magnet High School,93.9
6,Albert G Lane Technical High School,92.2
553,William Jones College Preparatory High School,90.1


### 3. CHICAGO CRIME Data

##### Finding the total number of crimes recorded in the CRIME table.


In [101]:
%sql select count(*) from CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


count(*)
533


List all data related to crimes involving minors?

In [106]:
df = %sql SELECT * FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%';
df = pd.DataFrame(df)
df

 * sqlite:///FinalDB.db
Done.


,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,3987219,HL266884,2005-03-31,024XX N CLARK ST,2210,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR,CONVENIENCE STORE,1,0,...,19,43.0,7.0,22,1172680.0,1916483.0,2005,41.926269,-87.640899,"(41.926268719, -87.640899336)"
1,3266814,HK238408,2004-03-13,093XX S STONY ISLAND AVE,2230,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR,ALLEY,1,0,...,4,8.0,48.0,22,1188539.0,1843379.0,2004,41.725301,-87.584966,"(41.72530099, -87.584965887)"


##### List all kidnapping crimes involving a child?

In [107]:
df = %sql SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE LIKE '%KIDNAPPING%';
df = pd.DataFrame(df)
df

 * sqlite:///FinalDB.db
Done.


,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,...,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.874908,-87.750249,"(41.874908413, -87.750249307)"


##### List the kind of crimes that were recorded at schools also which schools and description about crime.

In [121]:
%sql SELECT PRIMARY_TYPE ,DESCRIPTION ,LOCATION_DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION
BATTERY,SIMPLE,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING"
BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING"
BATTERY,SIMPLE,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL DAMAGE,TO VEHICLE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,POSS: HEROIN(WHITE),"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,MANU/DEL:CANNABIS 10GM OR LESS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,TO LAND,"SCHOOL, PUBLIC, GROUNDS"


##### Which community area is most crime prone? display via number

In [124]:
%sql select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA group by COMMUNITY_AREA_NUMBER order by count(*) desc limit 1;


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0
